In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
import random
%matplotlib inline

In [37]:
# Read in all the words
words = open('data/names.txt', 'r').read().splitlines()
print(words[:10])

# Build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)

print(f'Character to index mapping: {itos}')
print(f'Vocabulary size: {vocab_size}')

# Shuffle the words
random.seed(42)
random.shuffle(words)


['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']
Character to index mapping: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Vocabulary size: 27


In [39]:
# Encode the words with '.' so the model can learn when a name should end
def encode_words(words):
    encoded = []
    for w in words:
        encoded.extend([stoi[ch] for ch in '.' + w ])
    return encoded

encoded = encode_words(words)
encoded[:20]

[0, 25, 21, 8, 5, 14, 7, 0, 4, 9, 15, 14, 4, 18, 5, 0, 24, 1, 22, 9]

In [7]:
# Parameters for creating training and test sets
n = len(encoded)
n1 = int(0.8 * n)
block_size = 8
batch_size = 1

train_seq = encoded[:n1]
dev_seq = encoded[n1:]

In [8]:
def create_pairs(seq, block_size):
    X, Y = [], []
    for i in range(0, len(seq) - block_size, block_size):
        X.append(seq[i:i+block_size])
        Y.append(seq[i+1:i+block_size+1])
    X = torch.tensor(X, dtype=torch.float32)
    Y = torch.tensor(Y, dtype=torch.float32)
    return X, Y

Xtr, Ytr = create_pairs(train_seq, block_size)
Xdev, Ydev = create_pairs(dev_seq, block_size)

In [9]:
# We are sure that we can pass in previous example's hidden layer
print(Xtr[1])
print(Ytr[1])
print(Xtr[2])
print(Ytr[2])
print(Xtr[3])
print(Ytr[3])
print(f'Training data shapes - X: {Xtr.shape}, Y: {Ytr.shape}')
print(f'Development data shapes - X: {Xdev.shape}, Y: {Ydev.shape}')

tensor([ 4.,  9., 15., 14.,  4., 18.,  5.,  0.])
tensor([ 9., 15., 14.,  4., 18.,  5.,  0., 24.])
tensor([24.,  1., 22.,  9.,  5., 14.,  0., 10.])
tensor([ 1., 22.,  9.,  5., 14.,  0., 10., 15.])
tensor([15., 18.,  9.,  0., 10., 21.,  1., 14.])
tensor([18.,  9.,  0., 10., 21.,  1., 14., 12.])
Training data shapes - X: torch.Size([22814, 8]), Y: torch.Size([22814, 8])
Development data shapes - X: torch.Size([5703, 8]), Y: torch.Size([5703, 8])


In [20]:
lr = 0.0001
batch_size = 1
hidden_size = 30
time_steps = 8
input_size = 27

In [22]:
def softmax(x):
    e_x = torch.exp(x - torch.max(x, dim=1, keepdim=True)[0])
    return e_x / torch.sum(e_x, dim=1, keepdim=True)

In [185]:
# Initalize parameters
Fvh = torch.randn(vocab_size, hidden_size) * 0.01
i1vh = torch.randn(vocab_size, hidden_size) * 0.01
i2vh = torch.randn(vocab_size, hidden_size) * 0.01
Ovh = torch.randn(vocab_size, hidden_size) * 0.01

Fhh = torch.randn(hidden_size, hidden_size) * 0.01
i1hh = torch.randn(hidden_size, hidden_size) * 0.01
i2hh = torch.randn(hidden_size, hidden_size) * 0.01
Ohh = torch.randn(hidden_size, hidden_size) * 0.01

bias1 = torch.zeros(1, hidden_size)
bias2 = torch.zeros(1, hidden_size)
bias3 = torch.zeros(1, hidden_size)
bias4 = torch.zeros(1, hidden_size)

output_matrix = torch.randn(hidden_size, vocab_size) * 0.01
output_bias = torch.zeros(1, vocab_size)

X = {}
H = {}
C = {}
Ct = {}
preact1 = {}
preact2 = {}
preact3 = {}
preact4 = {}
act1 = {}
act2 = {}
act3 = {}
act4 = {}
logits = {}

In [ ]:
max_iterations = 0

while max_iterations < 100000:
    
    # Looping over batches (each batch is just a single row)
    for batch_num in range(Xtr.shape[0]):

        # Initial cell state and hidden state and loss
        H[-1] = torch.zeros(1, hidden_size)
        C[-1] = torch.zeros(1, hidden_size)
        loss = 0   
        
        # Get a batch of random numbers into correct shape
        Xb = Xtr[batch_num, :].to(torch.long) # (1, 8)
        Xb = F.one_hot(Xb, 27).float() # (8, 27)
        Yb = Ytr[batch_num].to(torch.long) # (8, 1)
        
### --------------------------------------------------------------------------------------------------------------        
        # Forward pass over all time steps
        for t in range(time_steps):
            preact1[t] = Xb[t] @ Fvh + H[t-1] @ Fhh + bias1 # (1, 27) @ (27, 30) + (1, 30) @ (30, 30) + (30) = (1, 30)
            preact2[t] = Xb[t]  @ i1vh + H[t-1]  @ i1hh * bias2 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            preact3[t] = Xb[t]  @ i2vh + H[t-1]  @ i2hh + bias3 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            preact4[t] = Xb[t]  @ Ovh + H[t-1]  @ Ohh + bias4 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
            
            act1[t] = torch.sigmoid(preact1[t]) # (1, 30)
            act2[t] = torch.sigmoid(preact2[t]) # (1, 30)
            act3[t] = torch.tanh(preact3[t]) # (1, 30)
            act4[t] = torch.sigmoid(preact4[t]) # (1, 30)

            # Forget gate * previous cell state + i1 gate * i2 gate
            C[t] = act1[t] * C[t-1]  + act2[t]  * act3[t]  # (1, 30)
            Ct[t] = torch.tanh(C[t]) # (1, 30)
            H[t] = Ct[t] * act4[t]  # (1, 30)          

            # Logits
            logits[t] = H[t]  @ output_matrix + output_bias # (1, 30) @ (30, 27) + (1, 27) = (1, 27)

            # Cross entropy
            counts = logits[t].exp()
            counts_sum = counts.sum(1, keepdims=True)
            counts_sum_inv = counts_sum**-1
            probs = counts * counts_sum_inv
            logprobs = probs.log()
            loss += -logprobs[0, Yb[t]]

        # Continue the sequence for time step -1 of the next input sequence
        H[-1] = H[time_steps - 1]
        C[-1] = C[time_steps - 1]

        # Print out the loss occasionally
        if max_iterations % 1000 == 0:
            print("loss", loss)
            print("iteration", max_iterations)
            sample_ix = sample(H[-1], C[-1], 2, 15)
            txt = ''.join(itos[ix] for ix in sample_ix)
            print('----\n %s \n----' % (txt,))

        # Increment max iterations
        max_iterations += 1 

### ---------------------------------------------------------------------------------------------------------------
        # Backward pass
        # Set to zeros initially
        dHnext = torch.zeros_like(H[0])
        dCnext = torch.zeros_like(C[0])
        
        dFvh = torch.zeros(vocab_size, hidden_size)
        di1vh = torch.zeros(vocab_size, hidden_size)
        di2vh = torch.zeros(vocab_size, hidden_size)
        dOvh = torch.zeros(vocab_size, hidden_size)
        
        dFhh = torch.zeros(hidden_size, hidden_size)
        di1hh = torch.zeros(hidden_size, hidden_size)
        di2hh = torch.zeros(hidden_size, hidden_size)
        dOhh = torch.zeros(hidden_size, hidden_size)
        
        dbias1 = torch.zeros(1, hidden_size)
        dbias2 = torch.zeros(1, hidden_size)
        dbias3 = torch.zeros(1, hidden_size)
        dbias4 = torch.zeros(1, hidden_size)
        
        doutput_matrix = torch.zeros(hidden_size, vocab_size)
        doutput_bias = torch.zeros(1, vocab_size)

        for t in reversed(range(time_steps)):
            # Backpropogate cross entropy
            dlogits = F.softmax(logits[t], 1)
            dlogits [0, Yb[t]] -= 1
            
            # Backpropogate output matrix and its bias
            doutput_matrix +=  H[t].T @ dlogits # (30, 1) @ (1, 27) = (30, 27)
            doutput_bias += dlogits # (1, 27)
            
            # Backpropogate into H
            dH = dlogits  @ output_matrix.T + dHnext # (1, 27) @ (27, 30) + (1, 30) = (1, 30)
            
            # Backpropogate dact4 (output gate activations)
            dact4  = dH  * Ct[t]  # (1, 30) * (1, 30) = (1, 30)
            
            # Backpropogate dC (current cell state)
            dC = dH * act4[t] * (1 - torch.tanh(C[t]) ** 2) + dCnext # (1, 30) * (1, 30) * (1, 30) = (1, 30)
            
            ### Backpropogate act1 and previous cell state
            # Forget gate activations
            # dCnext activations
            dact1 = dC * C[t-1] # (1, 30) * (1, 30) = (1, 30)
            dCnext = dC * act1[t]
            
            # Backpropogate i1 activations
            dact2  = dC  * act3[t] # (1, 27)
            
            # Backpropogate i2 activations
            dact3  = dC  * act2[t]  # (1, 27)
            
            # Backpropogate all preactivations
            dpreact1 = dact1 * act1[t] * (1 - act1[t])
            dpreact2 = dact2 * act2[t] * (1 - act2[t])
            dpreact3 = dact3 * (1 - torch.tanh(preact3[t]) ** 2)
            dpreact4 = dact4 * act4[t] * (1 - act4[t])
            
            # Backpropogate gates
            dFvh += Xb[t].view(vocab_size, 1) @ dpreact1  # (27, 1) (1, 30) = (27, 30)
            dFhh += H[t-1].view(hidden_size, 1) @ dpreact1  # (30, 1) (1, 30) = (30, 30)
            dbias1 += dpreact1
            
            di1vh += Xb[t].view(vocab_size, 1) @ dpreact2 # (27, 1) (1, 30) = (27, 30)
            di1hh += H[t-1].view(hidden_size, 1) @ dpreact2 # (30, 1) (1, 30) = (30, 30)
            dbias2 += dpreact2
            
            di2vh += Xb[t].view(vocab_size, 1) @ dpreact3 # (27, 1) (1, 30) = (27, 30)
            di2hh += H[t-1].view(hidden_size, 1) @ dpreact3 # (30, 1) (1, 30) = (30, 30)
            dbias3 += dpreact3
            
            dOvh += Xb[t].view(vocab_size, 1) @ dpreact4 # (27, 1) (1, 30) = (27, 30)
            dOhh += H[t-1].view(hidden_size, 1) @ dpreact4 # (30, 1) (1, 30) = (30, 30)
            dbias4 += dpreact4
            
            # Backpropogate dHnext
            # (1, 30) @ (30, 30) = (1, 30)
            dHnext  = dpreact1  @ Fhh.T + dpreact2  @ i1hh.T + dpreact3  @ i2hh.T + dpreact4  @ Ohh.T

        torch.nn.utils.clip_grad_norm_([dFvh, di1vh, di2vh, dOvh, dFhh, di1hh, di2hh, dOhh, dbias1, dbias2, dbias3, dbias4, doutput_matrix, doutput_bias], max_norm=1.0)

### ------------------------------------------------------------------------------------------------------------------
    
        # Update parameters using gradients
        Fvh -= lr * dFvh
        i1vh -= lr * di1vh
        i2vh -= lr * di2vh
        Ovh -= lr * dOvh

        Fhh -= lr * dFhh
        i1hh -= lr * di1hh
        i2hh -= lr * di2hh
        Ohh -= lr * dOhh
        
        bias1 -= lr * dbias1
        bias2 -= lr * dbias2
        bias3 -= lr * dbias3
        bias4 -= lr * dbias4

        output_matrix -= lr * doutput_matrix
        output_bias -= lr * doutput_bias
    
    if (max_iterations == 10000):
        break

    

loss tensor(26.3670)
iteration 0
----
 tbmsqqcxgdlfsbp 
----
loss tensor(26.1093)
iteration 1000
----
 pkgkfqcwimnudcd 
----
loss tensor(25.8782)
iteration 2000
----
 thkcuiotbwbelqq 
----


In [140]:
def sample(hprev, cprev, seed_ix, n):
    # Create one hot vector based on input seed_ix
    x = torch.zeros(1, vocab_size)
    x[0, seed_ix] = 1
    ixes = []

    for t in range(n):
        preact1 = x @ Fvh + hprev @ Fhh + bias1 # (1, 27) @ (27, 30) + (1, 30) @ (30, 30) + (30) = (1, 30)
        preact2 = x  @ i1vh + hprev  @ i1hh * bias2 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        preact3 = x  @ i2vh + hprev  @ i2hh + bias3 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        preact4 = x  @ Ovh + hprev  @ Ohh + bias4 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        
        act1 = torch.sigmoid(preact1) # (1, 30)
        act2 = torch.sigmoid(preact2) # (1, 30)
        act3 = torch.tanh(preact3) # (1, 30)
        act4 = torch.sigmoid(preact4) # (1, 30)

        # Forget gate * previous cell state + i1 gate * i2 gate
        C = act1 * cprev  + act2  * act3  # (1, 30)
        Ct = torch.tanh(C) # (1, 30)
        H = Ct * act4  # (1, 30)          

        # Logits
        logits = H  @ output_matrix + output_bias # (1, 30) @ (30, 27) + (1, 27) = (1, 27)

        # Probability
        counts = logits.exp()
        counts_sum = counts.sum(1, keepdims=True)
        counts_sum_inv = counts_sum**-1
        probs = counts * counts_sum_inv

        # Sample from distribution
        ix = torch.multinomial(probs, num_samples=1).item()
        if (ix == 0):
            break
        ixes.append(ix)

        # Update our parameters
        x = torch.zeros(1, vocab_size)
        x[0, ix] = 1
        hprev = H
        cprev = C
    return ixes

In [184]:
hprev = torch.zeros(1, hidden_size)
cprev = torch.zeros(1, hidden_size)

sample_ix = sample(hprev, cprev, 2, 15)
txt = ''.join(itos[ix] for ix in sample_ix)
print('----\n %s \n----' % (txt,))


----
 hlazsmbsaan 
----
